In [1]:
import requests
import datetime
from datetime import datetime as dt
import patoolib
import os
import pandas as pd
import sqlalchemy 
import psycopg2
from sqlalchemy import create_engine
import numpy as np
from datetime import timedelta

In [2]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [3]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [4]:
now = dt.now()
delta=0
status=400
while status!=200:
    date=now- timedelta(days=delta)
    print('testing date:'+str(date))
    dia=date.strftime("%d")
    mes=date.strftime("%m")
    year=str(date.year)
    url='http://deis.minsal.cl/wp-content/uploads/'+year+'/'+mes+'/DEFUNCIONES_FUENTE_DEIS_2016_'+year+'_'+dia+mes+year+'.rar'
    print('url: '+url)
    req = requests.get(url)
    status=req.status_code
    print(status)
    if status==200:
        print('success')
    else:
        print('failure, trying previous date')
        delta+=1

testing date:2020-12-06 23:59:14.129326
url: http://deis.minsal.cl/wp-content/uploads/2020/12/DEFUNCIONES_FUENTE_DEIS_2016_2020_06122020.rar
404
failure, trying previous date
testing date:2020-12-05 23:59:14.129326
url: http://deis.minsal.cl/wp-content/uploads/2020/12/DEFUNCIONES_FUENTE_DEIS_2016_2020_05122020.rar
404
failure, trying previous date
testing date:2020-12-04 23:59:14.129326
url: http://deis.minsal.cl/wp-content/uploads/2020/12/DEFUNCIONES_FUENTE_DEIS_2016_2020_04122020.rar
404
failure, trying previous date
testing date:2020-12-03 23:59:14.129326
url: http://deis.minsal.cl/wp-content/uploads/2020/12/DEFUNCIONES_FUENTE_DEIS_2016_2020_03122020.rar
200
success


In [5]:
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [6]:
open(path+'DEIS.rar', 'wb').write(req.content)

4783749

In [7]:
if not os.path.exists(path+'DEIS'+timestamp):
    os.makedirs(path+'DEIS'+timestamp)
    patoolib.extract_archive(path+'DEIS.rar', outdir=path+'DEIS'+timestamp)

patool: Extracting /data/ETLcache/DEIS.rar ...
patool: running /usr/bin/rar x -- /data/ETLcache/DEIS.rar
patool:     with cwd='/data/ETLcache/DEIS_06122020_235946'
patool: ... /data/ETLcache/DEIS.rar extracted to `/data/ETLcache/DEIS_06122020_235946'.


In [8]:
for f in os.listdir(path+'DEIS'+timestamp):
    if '.csv' in f:
        deis=pd.read_csv(path+'DEIS'+timestamp+'/'+f,sep=';',encoding='latin1',header=0)
    if '.xlsx' in f:
        headers=pd.read_excel(path+'DEIS'+timestamp+'/'+f,header=3,index_col=1)

In [9]:
deis

,2020,2020-11-10,Hombre,1,114,13114,Las Condes,Metropolitana de Santiago,I259,I00-I99,...,ENFERMEDAD ISQUEMICA CRONICA DEL CORAZÓN NO ESPECIFICADA,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,2020,2020-07-21,Hombre,1,75,6115,Rengo,Del Libertador B. O'Higgins,I678,I00-I99,...,OTRAS ENFERMEDADES CEREBROVASCULARES ESPECIFIC...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,2020-02-13,Hombre,1,92,5101,Valparaíso,De Valparaíso,F03X,F00-F99,...,DEMENCIA NO ESPECIFICADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,2020-01-31,Mujer,1,85,13130,San Miguel,Metropolitana de Santiago,J189,J00-J99,...,NEUMONIA NO ESPECIFICADA,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,2020-10-13,Hombre,1,91,7101,Talca,Del Maule,U072,U00-U99,...,COVID-19 virus no identificado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,2020-09-12,Hombre,1,89,13109,La Cisterna,Metropolitana de Santiago,U072,U00-U99,...,COVID-19 virus no identificado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542708,2016,2016-08-28,Mujer,1,74,10103,Cochamó,De Los Lagos,E119,E00-E90,...,DIABETES MELLITUS NO INSULINODEPENDIENTE SIN M...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
542709,2016,2016-08-30,Mujer,1,86,10209,Quemchi,De Los Lagos,J449,J00-J99,...,ENFERMEDAD PULMÓNAR OBSTRUCTIVA CRONICA NO ESP...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
542710,2016,2016-08-26,Hombre,1,84,10205,Dalcahue,De Los Lagos,C61X,C00-D48,...,TUMOR MALIGNO DE LA PRÓSTATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
542711,2016,2016-09-02,Mujer,1,90,10205,Dalcahue,De Los Lagos,J459,J00-J99,...,ASMA NO ESPECIFICADO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
headers=headers['Nombre de la variable']

In [11]:
#headers=headers[1:]
deis.columns=headers

In [13]:
deis['Metropolitana']=0
deis.loc[deis['GLOSA_REG_RES']=='Metropolitana de Santiago','Metropolitana']=1
deis['Metropolitana']=deis['Metropolitana'].replace({0:'Resto de Chile',1:'Región Metropolitana'})

In [19]:
deis['FECHA_DEF']=pd.to_datetime(deis['FECHA_DEF'],format='%Y-%m-%d',errors='coerce')
deis['FECHA_DEF']=deis['FECHA_DEF'].dt.to_pydatetime()

In [21]:
deis['FECHA_STR']=deis['FECHA_DEF'].dt.strftime('%d-%m-%Y')

In [22]:
deis['COVID']=deis['DIAG1'].replace({'U071':'Covid PCR+','U072':'Covid Sospechoso'})
deis.loc[~deis['DIAG1'].isin(['U071','U072']),'COVID']=np.nan

In [23]:
deis['COVID'].value_counts()

Covid PCR+          15592
Covid Sospechoso     5175
Name: COVID, dtype: int64

In [24]:
if not 'ANO_DEF' in deis.columns:
    print('creando columna: ANO_DEF')
    deis['ANO_DEF']=deis['FECHA_DEF'].dt.year

In [25]:
for k in ['DEIS']:
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema DEIS exists, will not be created


In [26]:
edad_fecha=deis.loc[(deis['ANO_DEF']==2020) & (deis['COVID'].notnull())]
edad_fecha=edad_fecha.groupby(['FECHA_DEF','EDAD_CANT'])['COVID'].count().reset_index()
edad_fecha=edad_fecha.set_index('FECHA_DEF').groupby('EDAD_CANT').rolling('7d',min_periods=1)['COVID'].mean()
edad_fecha=edad_fecha.reset_index()
edad_fecha.to_sql('DEIS_EDAD_FECHA_ROLLING', schema='DEIS',con=cnx,if_exists='replace')

In [27]:
schema='DEIS'
for y in deis['ANO_DEF'].unique():
    name='DEIS_'+str(y).split('.')[0]
    df=deis[deis['ANO_DEF']==y]
    print("creating table "+name+' ,schema: '+schema)
    df.to_sql(name, schema=schema,con=cnx,if_exists='replace')
    print("saving table"+path+name+timestamp+'.csv in cache')
    df.to_csv(path+name+timestamp+'.csv',encoding='utf-8')

creating table DEIS_2020 ,schema: DEIS
saving table/data/ETLcache/DEIS_2020_06122020_235946.csv in cache
creating table DEIS_2019 ,schema: DEIS
saving table/data/ETLcache/DEIS_2019_06122020_235946.csv in cache
creating table DEIS_2018 ,schema: DEIS
saving table/data/ETLcache/DEIS_2018_06122020_235946.csv in cache
creating table DEIS_2017 ,schema: DEIS
saving table/data/ETLcache/DEIS_2017_06122020_235946.csv in cache
creating table DEIS_2016 ,schema: DEIS
saving table/data/ETLcache/DEIS_2016_06122020_235946.csv in cache


In [35]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_covid-chile.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/covid-chile.ipynb')

65280